In [1]:
##### add python path #####
import sys
import os
PATH = os.getcwd()
for dir_idx, dir_name in enumerate(PATH.split('/')):
    if 'optimal_control' in dir_name.lower():
        PATH = '/'.join(PATH.split('/')[:(dir_idx+1)])
        break
if not PATH in sys.path:
    sys.path.append(PATH)
###########################

from trajectory_planner import TrajectoryPlanner
from foot_step_planner import FootStepPlanner
from gait_scheduler import GaitScheduler
from env.quadruped.env import Env

from scipy.spatial.transform import Rotation
from scipy.optimize import minimize
from scipy.optimize import Bounds
from copy import deepcopy
import numpy as np
import time

In [2]:
env = Env(enable_draw=True, base_fix=False)
num_leg = env.num_leg
time_step = env.time_step
base_inertia = deepcopy(env.model.inertia)
base_mass = env.model.mass
friction_coef = env.friction_coef

plan_time_step = 0.05
plan_time_horizon = 10

gs_args = dict()
gs_args['num_leg'] = num_leg
gs_args['gait_period'] = 1.0
gs_args['is_visualize'] = False
gait_scheduler = GaitScheduler(gs_args)

tp_args = dict()
tp_args['time_step'] = plan_time_step
tp_args['time_horizon'] = plan_time_horizon
tp_args['max_accel'] = 0.1
tp_args['max_ang_accel'] = 0.1
trajectory_planner = TrajectoryPlanner(tp_args)

fsp_args = dict()
fsp_args['num_leg'] = num_leg
fsp_args['time_horizon'] = plan_time_horizon
fsp_args['abduct_org_list'] = env.model.abduct_org_list
foot_step_planner = FootStepPlanner(fsp_args)

In [3]:
env.reset()
gait_scheduler.reset()
trajectory_planner.reset()
foot_step_planner.reset()

In [4]:
'''
gait_list = np.zeros((num_leg, 2)) # FR, FL, RR, RL
gait_list[0,0] = 0.5 # offset
gait_list[0,1] = 0.5 # duration
gait_list[2,0] = 0.0 # offset
gait_list[2,1] = 0.5 # duration
gait_scheduler.gait_list = deepcopy(gait_list)
gait_scheduler.update()
'''

vel_cmd = np.zeros(3)
vel_cmd[0] = 0.2
vel_cmd[1] = 0.0
ang_vel_cmd = np.zeros(3)
ang_vel_cmd[2] = 0.0
height_cmd = 0.35

In [5]:
init_state_list = np.concatenate([env.model.com_pos, env.model.base_rpy, env.model.base_vel, env.model.base_ang_vel])
init_foot_pos_list = deepcopy(env.model.foot_pos_list)

target_trajectory_list = trajectory_planner.get_target_trajectory_list(vel_cmd, ang_vel_cmd, height_cmd, init_state_list)
contact_list, step_info_list = gait_scheduler.get_contact_list(plan_time_step, plan_time_horizon)
target_foot_step_list = foot_step_planner.get_target_foot_step_list(init_foot_pos_list, step_info_list, target_trajectory_list, \
                                                                    vel_cmd, ang_vel_cmd, height_cmd)

In [6]:
def obj_func(x, args):
    target_trajectory_list = args[0]
    Q_mat = args[1] # 12 x 12
    R_mat = args[2] # 12 x 12

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    difference_list = state_list - target_trajectory_list # N x 12
    f = 0.5*np.sum(np.matmul(difference_list, Q_mat)*difference_list)
    f += 0.5*np.sum(np.matmul(force_list, R_mat)*force_list)
    return f

def obj_jacobian(x, args):
    target_trajectory_list = args[0]
    Q_mat = args[1] # 12 x 12
    R_mat = args[2] # 12 x 12

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))

    jacobian = np.zeros(len(x))

    difference_list = state_list - target_trajectory_list # N x 12
    jacobian[:12*time_horizon] = np.matmul(difference_list, Q_mat).ravel()
    jacobian[12*time_horizon:] = np.matmul(force_list, R_mat).ravel()
    return jacobian

In [7]:
def get_cross_product_matrix(vector):
    a, b, c = vector
    mat = np.array([[0, -c, b],
                   [c, 0, -a],
                   [-b, a, 0]])
    return mat

In [8]:
def eq_cons_func(x, args):
    init_state_list = args[0]
    target_foot_step_list = args[1]
    contact_list = args[2]
    delta_t = args[3]
    base_inertia = args[4]
    base_mass = args[5]

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    eq_cons = []
    state = init_state_list
    gravity = np.zeros(12)
    gravity[6:9] = np.array([0, 0, -9.8])
    for time_idx in range(time_horizon):
        next_state = state_list[time_idx]
        force = (force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)).ravel()

        yaw = state[5]
        r = Rotation.from_euler('z', yaw, degrees=False)
        yaw_mat = r.as_matrix()
        global_inertia = np.matmul(yaw_mat, np.matmul(base_inertia, yaw_mat.T))
        inv_global_inertia = np.linalg.inv(global_inertia)
        foot_step_list = target_foot_step_list[time_idx]

        A_mat = np.eye(12)
        A_mat[0:3, 6:9] = delta_t*np.eye(3)
        A_mat[3:6, 9:12] = delta_t*yaw_mat

        B_mat = np.zeros((12, 12))
        for leg_idx in range(num_leg):
            foot_step = foot_step_list[leg_idx]
            B_mat[6:9, leg_idx*3:(leg_idx+1)*3] = np.eye(3)*(delta_t/base_mass)
            B_mat[9:12, leg_idx*3:(leg_idx+1)*3] = delta_t*np.matmul(inv_global_inertia, get_cross_product_matrix(foot_step))
        
        eq_cons.append((np.matmul(A_mat, state) + np.matmul(B_mat, force) + gravity - next_state).ravel())
        state = next_state

    eq_cons = np.concatenate(eq_cons)
    return eq_cons

def eq_cons_jacobian(x, args):
    init_state_list = args[0]
    target_foot_step_list = args[1]
    contact_list = args[2]
    delta_t = args[3]
    base_inertia = args[4]
    base_mass = args[5]

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    eq_cons_jacobian = np.zeros((12*time_horizon, len(x)))
    state = init_state_list
    for time_idx in range(time_horizon):        
        next_state = state_list[time_idx]
        force = (force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)).ravel()

        yaw = state[5]
        wx, wy, wz = state[9:12]
        r = Rotation.from_euler('z', yaw, degrees=False)
        yaw_mat = r.as_matrix()
        global_inertia = np.matmul(yaw_mat, np.matmul(base_inertia, yaw_mat.T))
        inv_global_inertia = np.linalg.inv(global_inertia)
        inv_base_inertia = np.linalg.inv(base_inertia)
        foot_step_list = target_foot_step_list[time_idx]

        A_mat = np.eye(12)
        A_mat[0:3, 6:9] = delta_t*np.eye(3)
        A_mat[3:6, 9:12] = delta_t*yaw_mat

        B_mat = np.zeros((12, 12))
        for leg_idx in range(num_leg):
            foot_step = foot_step_list[leg_idx]
            B_mat[6:9, leg_idx*3:(leg_idx+1)*3] = np.eye(3)*(delta_t/base_mass)
            B_mat[9:12, leg_idx*3:(leg_idx+1)*3] = delta_t*np.matmul(inv_global_inertia, get_cross_product_matrix(foot_step))
        
        if time_idx > 0:
            eq_cons_jacobian[time_idx*12:(time_idx+1)*12, (time_idx-1)*12:time_idx*12] = A_mat
        temp_mat = np.zeros((12,12))
        temp_mat[3,5] = -delta_t*(wx*np.sin(yaw) + wy*np.cos(yaw))
        temp_mat[3,6] = delta_t*(wx*np.cos(yaw) - wy*np.sin(yaw))
        eq_cons_jacobian[time_idx*12:(time_idx+1)*12, time_idx*12:(time_idx+1)*12] += deepcopy(temp_mat)
        
        temp_mat = np.zeros((12,12))
        derivative_R = np.array([[-np.sin(yaw), -np.cos(yaw), 0],[np.cos(yaw), -np.sin(yaw), 0],[0, 0, 0]])
        a = np.zeros(3)
        for leg_idx in range(num_leg):
            foot_step = foot_step_list[leg_idx]
            a += delta_t*np.cross(foot_step, force[leg_idx*3:(leg_idx+1)*3])
        temp_mat[9:12, 5] += np.matmul(np.matmul(derivative_R, np.matmul(inv_base_inertia, yaw_mat)), a)
        temp_mat[9:12, 5] += np.matmul(np.matmul(yaw_mat, np.matmul(inv_base_inertia, derivative_R)), a)
        eq_cons_jacobian[time_idx*12:(time_idx+1)*12, time_idx*12:(time_idx+1)*12] += deepcopy(temp_mat)
        eq_cons_jacobian[time_idx*12:(time_idx+1)*12, time_idx*12:(time_idx+1)*12] -= np.eye(12)

        eq_cons_jacobian[time_idx*12:(time_idx+1)*12, 12*(time_horizon + time_idx):12*(time_horizon + time_idx + 1)] = B_mat

        state = next_state
    return eq_cons_jacobian

eq_cons = {
    'type':'eq',
    'fun' :eq_cons_func,
    'jac' :eq_cons_jacobian,
    'args':[[init_state_list, target_foot_step_list, contact_list, plan_time_step, base_inertia, base_mass]],
    }

In [9]:
def ineq_cons_func(x, args):
    contact_list = args[0]
    friction_coef = args[1]

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    ineq_cons = []
    for time_idx in range(time_horizon):
        force = force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)
        ineq_cons.append(-abs(force[:,0]) + friction_coef*force[:,2])
        ineq_cons.append(-abs(force[:,1]) + friction_coef*force[:,2])
        ineq_cons.append(force[:,2])

    ineq_cons = np.concatenate(ineq_cons)
    return ineq_cons

def ineq_cons_jacobian(x, args):
    contact_list = args[0]
    friction_coef = args[1]

    time_horizon = int(len(x)/24)
    state_list = x[:int(len(x)/2)].reshape((time_horizon, 12))
    force_list = x[int(len(x)/2):].reshape((time_horizon, 12))
    
    ineq_cons_jacobian = np.zeros((12*time_horizon, len(x)))
    for time_idx in range(time_horizon):
        force = force_list[time_idx].reshape((4,3))*contact_list[time_idx].reshape(4,1)
        for i in range(4):
            ineq_cons_jacobian[12*time_idx + i, 12*(time_horizon + time_idx) + i*3] = 1 if force[i,0] < 0 else -1
            ineq_cons_jacobian[12*time_idx + i, 12*(time_horizon + time_idx) + i*3 + 2] = friction_coef
        for i in range(4):
            ineq_cons_jacobian[12*time_idx + 4 + i, 12*(time_horizon + time_idx) + i*3 + 1] = 1 if force[i,1] < 0 else -1
            ineq_cons_jacobian[12*time_idx + 4 + i, 12*(time_horizon + time_idx) + i*3 + 2] = friction_coef
        for i in range(4):
            ineq_cons_jacobian[12*time_idx + 8 + i, 12*(time_horizon + time_idx) + i*3 + 2] = 1
    return ineq_cons_jacobian

ineq_cons = {
    'type':'ineq',
    'fun' :ineq_cons_func,
    'jac' :ineq_cons_jacobian,
    'args':[[contact_list, friction_coef]],
    }

In [10]:
init_force_list = np.zeros((plan_time_horizon, 12))
init_x_list = np.concatenate([target_trajectory_list.ravel(), init_force_list.ravel()])

In [11]:
max_force = 1000.0
lowers = np.array([-np.inf]*int(len(init_x_list)/2) + [-max_force]*int(len(init_x_list)/2))
uppers = np.array([np.inf]*int(len(init_x_list)/2) + [max_force]*int(len(init_x_list)/2))
bounds = Bounds(lowers, uppers)

In [12]:
Q_mat = np.eye(12) #np.diag([])
R_mat = 0.001*np.eye(12) #np.diag([])

In [25]:
res = minimize(obj_func, init_x_list, args=[target_trajectory_list, Q_mat, R_mat], \
               method="SLSQP", jac=obj_jacobian, bounds=bounds, constraints=[eq_cons, ineq_cons], \
               options={'ftol':1e-5, 'disp':True, 'maxiter':100, 'eps':1e-10})

Optimization terminated successfully    (Exit mode 0)
            Current function value: 2889.613868385557
            Iterations: 41
            Function evaluations: 42
            Gradient evaluations: 41


In [14]:
eq_cons_value = eq_cons_func(res.x, *eq_cons['args'])
eq_cons_value*(eq_cons_value > 0.001)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        0.,  0.,  0.,  0., -0.,  0., -0.,  0., -0., -0.,  0., -0., -0.,
        0.,  0.,  0., -0.,  0., -0.,  0.,  0., -0.,  0.,  0., -0.,  0.,
        0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0.,  0.,  0., -0.,
       -0.,  0., -0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,  0., -0.,
        0., -0.,  0., -0., -0.,  0., -0., -0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0., -0., -0.,  0.,  0.,  0., -0., -0.,  0.,  0.,  0.,
        0., -0.,  0.,  0.,  0., -0.,  0., -0.,  0.,  0.,  0., -0., -0.,
        0., -0., -0.])

In [15]:
ineq_cons_value = ineq_cons_func(res.x, *ineq_cons['args'])
ineq_cons_value

array([385.87209785, 385.87209785, 364.08116482, 364.08116482,
       386.0935317 , 386.0935317 , 364.30259867, 364.30259867,
       428.992813  , 428.992813  , 404.78066519, 404.78066519,
       382.01715898, 382.01715898, 360.44433051, 360.44433051,
       382.23374416, 382.23374416, 360.66091568, 360.66091568,
       424.70416017, 424.70416017, 400.73435076, 400.73435076,
       375.64123894, 375.64123894, 354.42743603, 354.42743603,
       375.852208  , 375.852208  , 354.63840509, 354.63840509,
       417.61356445, 417.61356445, 394.04267232, 394.04267232,
       366.03354655, 366.03354655, 345.3616429 , 345.3616429 ,
       366.23759878, 366.23759878, 345.56569513, 345.56569513,
       406.93066531, 406.93066531, 383.96188348, 383.96188348,
       352.06021796, 352.06021796, 332.18475644, 332.18475644,
       352.25532316, 352.25532316, 332.37986165, 332.37986165,
       391.39480352, 391.39480352, 369.31095739, 369.31095739,
       331.98447256, 331.98447256, 313.2467102 , 313.24

In [16]:
cube = env.pybullet_client.loadURDF("cube_small.urdf")

In [17]:
collisionFilterGroup = 0x2
collisionFilterMask = 0x2
for i in range(env.pybullet_client.getNumJoints(env.model.sim_model)):
    env.pybullet_client.setCollisionFilterGroupMask(env.model.sim_model, i, collisionFilterGroup, collisionFilterMask)

for i in range(env.pybullet_client.getNumJoints(cube)):
    env.pybullet_client.setCollisionFilterGroupMask(cube, i, collisionFilterGroup, collisionFilterMask)

collisionFilterGroup2 = 0x1
collisionFilterMask2 = 0x1 
env.pybullet_client.setCollisionFilterGroupMask(env.planeId, -1, collisionFilterGroup2, collisionFilterMask2)

In [26]:
i = 0

In [34]:
state = res.x[i*12:(i+1)*12]
pos_base = state[:3]
rpy_base = state[3:6]
vel_base = state[6:9]
ang_vel_base = state[9:]
orn_base = env.pybullet_client.getQuaternionFromEuler(rpy_base)

env.pybullet_client.resetBasePositionAndOrientation(cube, pos_base, orn_base)
env.pybullet_client.resetBaseVelocity(cube, np.zeros(3), np.zeros(3))

print(pos_base, target_trajectory_list[i,:3])

i += 1

[ 1.15930487e-02  3.63860820e-15 -7.27188716e-01] [0.014 0.    0.35 ]


In [20]:
force_result = res.x[12*plan_time_horizon:].reshape((plan_time_horizon, 4, 3))

In [21]:
force_result

array([[[ 2.21433850e-01,  2.15384413e-13,  4.28992813e+02],
        [ 2.21433850e-01,  2.14714492e-13,  4.28992813e+02],
        [ 2.21433850e-01,  6.06220005e-14,  4.04780665e+02],
        [ 2.21433850e-01,  8.79776036e-14,  4.04780665e+02]],

       [[ 2.16585175e-01,  1.75194443e-13,  4.24704160e+02],
        [ 2.16585175e-01,  1.76268787e-13,  4.24704160e+02],
        [ 2.16585175e-01,  8.71633249e-14,  4.00734351e+02],
        [ 2.16585175e-01,  1.96256518e-13,  4.00734351e+02]],

       [[ 2.10969060e-01, -7.19357587e-14,  4.17613564e+02],
        [ 2.10969060e-01, -7.28166087e-14,  4.17613564e+02],
        [ 2.10969060e-01,  3.34123695e-13,  3.94042672e+02],
        [ 2.10969060e-01,  2.38643161e-13,  3.94042672e+02]],

       [[ 2.04052227e-01,  2.30266171e-13,  4.06930665e+02],
        [ 2.04052227e-01,  2.18308647e-13,  4.06930665e+02],
        [ 2.04052227e-01, -4.87556801e-15,  3.83961883e+02],
        [ 2.04052227e-01, -9.97537146e-14,  3.83961883e+02]],

       [[ 1.9510